In [1]:
import os
import pandas as pd
import numpy as np
import itertools
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.linear_model import LinearRegression

2023-10-06 14:36:58.451600: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-06 14:36:58.493116: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-06 14:36:58.493636: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-06 14:36:59.249370: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
DATA_DIR = './dms_data'
N_TRIALS = 50

In [3]:

def load_and_preprocess_data(file_path):
    """CSVを読み込んで前処理する関数。"""
    df = pd.read_csv(file_path)
    df = df.drop(['timestamp'], axis=1)
    return df.dropna()

def get_data_from_directory(directory_path):
    """指定ディレクトリのCSV全部を読み込んで結合する関数。"""
    files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
    data_frames = [load_and_preprocess_data(os.path.join(directory_path, file)) for file in files]
    return pd.concat(data_frames, ignore_index=True)

# トレインデータとテストデータを読み込む
train = get_data_from_directory(os.path.join(DATA_DIR, 'train'))
test = get_data_from_directory(os.path.join(DATA_DIR, 'test'))

# 特徴量を定義
features = [
    'm_speed', 'm_speed_var_480', 'm_speed_stddev_480', 'm_acceleration',
    'm_acceleration_var_480', 'm_acceleration_stddev_480', 'm_jerk',
    'm_jerk_var_480', 'm_jerk_stddev_480'
]
    
def evaluate_model(model, X, y):
    """モデルの評価。RMSE"""
    predictions = model.predict(X)
    return np.sqrt(mean_squared_error(y, predictions))

def printResults(rmse_results,title):
    mean_rmse = np.mean(rmse_results)
    median_rmse = np.median(rmse_results)
    variance_rmse = np.var(rmse_results)
    std_rmse = np.std(rmse_results)
    min_rmse = np.min(rmse_results)
    max_rmse = np.max(rmse_results)

    # 統計量を表示
    print(f"[{title}]RMSEの平均値: {mean_rmse}")
    print(f"[{title}]RMSEの中央値: {median_rmse}")
    print(f"[{title}]RMSEの分散: {variance_rmse}")
    print(f"[{title}]RMSEの標準偏差: {std_rmse}")
    print(f"[{title}]RMSEの最小値: {min_rmse}")
    print(f"[{title}]RMSEの最大値: {max_rmse}")

In [8]:
#print(train_data.head())
#print(train_data.describe())

data_dir = DATA_DIR  # データのディレクトリパス
data_lengths = []

# `test` と `train` ディレクトリ内の各CSVファイルのデータ長を確認
for dataset_type in ['test', 'train']:
    dataset_path = os.path.join(data_dir, dataset_type)
    csv_files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]
    for csv_file in csv_files:
        csv_path = os.path.join(dataset_path, csv_file)
        df = pd.read_csv(csv_path)
        data_lengths.append((dataset_type, csv_file, len(df)))
print(data_lengths)
# データの長さが最小のものを確認
min_length_data = min(data_lengths, key=lambda x: x[2])
min_length_data



[('test', '20201126_1546_0_y_test.csv', 36), ('test', '20201127_1432_7_y_test.csv', 35), ('test', '20201127_1548_2_y_test.csv', 36), ('test', '20201127_1701_7_y_test.csv', 35), ('test', '20201127_1840_5_y_test.csv', 36), ('test', '20201130_1122_5_y_test.csv', 37), ('test', '20201130_1808_6_y_test.csv', 37), ('test', '20201201_1230_0_y_test.csv', 37), ('test', '20201201_1429_5_y_test.csv', 36), ('test', '20201201_1555_0_y_test.csv', 36), ('test', '20201203_1022_7_y_test.csv', 36), ('test', '20201203_1244_5_y_test.csv', 36), ('test', '20201203_1404_6_y_test.csv', 36), ('test', '20201210_1112_2_y_test.csv', 35), ('test', '20201210_1354_2_y_test.csv', 35), ('test', '20201210_1610_6_y_test.csv', 38), ('train', '20201126_1546_0_y_train.csv', 144), ('train', '20201127_1432_7_y_train.csv', 141), ('train', '20201127_1548_2_y_train.csv', 146), ('train', '20201127_1701_7_y_train.csv', 141), ('train', '20201127_1840_5_y_train.csv', 145), ('train', '20201130_1122_5_y_train.csv', 151), ('train', '20

('test', '20201127_1432_7_y_test.csv', 35)